In [25]:
import mne
import numpy as np
from mne.datasets import sample

In [26]:
data_path = sample.data_path()
raw_fname = data_path + '/MEG/sample/sample_audvis_raw.fif'
info = mne.io.read_info(raw_fname) 

    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle


In [27]:
raw_fname

'/Users/DanielLongo/mne_data/MNE-sample-data/MEG/sample/sample_audvis_raw.fif'

In [28]:
fwd = mne.read_forward_solution("sample_forward_model")

Reading forward solution from sample_forward_model...
    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523) not available
    Read EEG forward solution (7498 sources, 60 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame


<ipython-input-28-fdf35a2566d0>:1: RuntimeWarning: This filename (sample_forward_model) does not conform to MNE naming conventions. All forward files should end with -fwd.fif, -fwd.fif.gz, _fwd.fif or _fwd.fif.gz
  fwd = mne.read_forward_solution("sample_forward_model")


In [29]:
fwd

<Forward | MEG channels: 0 | EEG channels: 60 | Source space: Surface with 7498 vertices | Source orientation: Free>

In [30]:
fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                         use_cps=True)

    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]


In [39]:
leadfield = fwd_fixed['sol']['data']
print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

Leadfield size : 60 sensors x 7498 dipoles


In [40]:
n_dipoles = leadfield.shape[1]
vertices = [src_hemi['vertno'] for src_hemi in fwd_fixed['src']]
stc = mne.SourceEstimate(1e-9 * np.eye(n_dipoles), vertices, tmin=0., tstep=1)
leadfield = mne.apply_forward(fwd_fixed, stc, info).data / 1e-9

Projecting source estimate to sensor space...
[done]


In [56]:
n_dipoles = leadfield.shape[1]
vertices = [src_hemi['vertno'] for src_hemi in fwd_fixed['src']]
z = np.random.randn(n_dipoles, n_dipoles)
stc = mne.SourceEstimate(z, vertices, tmin=0., tstep=1)
leadfield = mne.apply_forward(fwd_fixed, stc, info).data / np.sum(z, axis=1)

Projecting source estimate to sensor space...
[done]


<ipython-input-56-4995ad19aaa5>:5: RuntimeWarning: The maximum current magnitude is 5643562683.4 nAm, which is very large. Are you trying to apply the forward model to noise-normalized (dSPM, sLORETA, or eLORETA) values? The result will only be correct if currents (in units of Am) are used.
  leadfield = mne.apply_forward(fwd_fixed, stc, info).data / np.sum(z, axis=1)


In [58]:
np.eye(n_dipoles)

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [45]:
np.random.randn(n_dipoles, n_dipoles).shape

(7498, 7498)

In [57]:
np.sum(leadfield)

48732352.833470516

In [53]:
np.sum(z, axis=1).shape

(7498,)